In [29]:
import pandas as pd
from matplotlib import pyplot as plt
import mysql.connector
import folium

In [30]:
# AWS INSTANCE에서 데이터 가져오기

yespa_db = mysql.connector.connect(
    host = 'database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'eda_team',
    database = 'eda_project',
    password = '***'
)

cur = yespa_db.cursor(buffered=True, dictionary=True)

cur.execute('SELECT * FROM seoul_metro_coor')
result = cur.fetchall()

for data in result:
    print(data)
    
yespa_db.close()

{'역명': '미사', '호선': '5호선', '위도': 37.560927, '경도': 127.193877}
{'역명': '강일', '호선': '5호선', '위도': 37.55749, '경도': 127.17593}
{'역명': '김포공항', '호선': '김포골드라인', '위도': 37.56236, '경도': 126.801868}
{'역명': '고촌', '호선': '김포골드라인', '위도': 37.601243, '경도': 126.770345}
{'역명': '풍무', '호선': '김포골드라인', '위도': 37.612488, '경도': 126.732387}
{'역명': '사우', '호선': '김포골드라인', '위도': 37.620249, '경도': 126.719731}
{'역명': '걸포북변', '호선': '김포골드라인', '위도': 37.63165, '경도': 126.705975}
{'역명': '운양', '호선': '김포골드라인', '위도': 37.653867, '경도': 126.68393}
{'역명': '장기', '호선': '김포골드라인', '위도': 37.643986, '경도': 126.669017}
{'역명': '마산', '호선': '김포골드라인', '위도': 37.640732, '경도': 126.644344}
{'역명': '구래', '호선': '김포골드라인', '위도': 37.645384, '경도': 126.628633}
{'역명': '양촌', '호선': '김포골드라인', '위도': 37.642379, '경도': 126.614309}
{'역명': '원시', '호선': '서해선', '위도': 37.302371, '경도': 126.786691}
{'역명': '원곡', '호선': '서해선', '위도': 37.31321, '경도': 126.796261}
{'역명': '초지', '호선': '서해선', '위도': 37.319619, '경도': 126.808147}
{'역명': '선부', '호선': '서해선', '위도': 37.334353, '경도': 126.8099

In [31]:
df = pd.DataFrame(result)
df

,역명,호선,위도,경도
0,미사,5호선,37.560927,127.193877
1,강일,5호선,37.557490,127.175930
2,김포공항,김포골드라인,37.562360,126.801868
3,고촌,김포골드라인,37.601243,126.770345
4,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...
766,종로5가,1호선,37.570926,127.001849
767,종로3가,1호선,37.570406,126.991847
768,종각,1호선,37.570161,126.982923
769,시청,1호선,37.565715,126.977088


In [32]:
df_standard = df[df['역명'] == '가산디지털단지']
df_standard

,역명,호선,위도,경도
245,가산디지털단지,7호선,37.480338,126.882656
518,가산디지털단지,경부선,37.481581,126.882581


In [33]:
# ?? 왜 경부선? 위치는 맞음
df_standard.loc[df_standard['호선'] == '경부선', '호선'] = '1호선'
df_standard

,역명,호선,위도,경도
245,가산디지털단지,7호선,37.480338,126.882656
518,가산디지털단지,1호선,37.481581,126.882581


In [34]:
df_standard.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 245 to 518
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      2 non-null      object 
 1   호선      2 non-null      object 
 2   위도      2 non-null      float64
 3   경도      2 non-null      float64
dtypes: float64(2), object(2)
memory usage: 80.0+ bytes


In [35]:
df_standard.reset_index(drop=True, inplace=True)

In [36]:
df_standard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      2 non-null      object 
 1   호선      2 non-null      object 
 2   위도      2 non-null      float64
 3   경도      2 non-null      float64
dtypes: float64(2), object(2)
memory usage: 192.0+ bytes


In [37]:
route_colors_data = {
    "노선": ["1호선", "2호선", "3호선", "4호선", "5호선", "6호선", "7호선", "8호선", "9호선", "경의중앙선", "경춘선", "경원선", "경부선", "경인선", "경의선", "공항철도", "과천선", "분당선", "수인선", "안산선", "우이신설선", "일산선", "장항선", "중앙선","경강선"],
    "노선색": ["#003DA5", "#83B600", "#E51837", "#009C3A", "#FFD900", "#8B50A4", "#F05A28", "#00A3E0", "#AA9872", "#F5A200", "#CD7C2F", "#FFD700", "#009D3E", "#0065A4", "#B7D968", "#00A3E0", "#009D3E", "#FF6C00", "#00A3E0", "#A2006D", "#B8B8B8", "#84B817", "#FFE616", "#00A3E0", "#E60012"]
}

# 데이터프레임 생성
route_colors_df = pd.DataFrame(route_colors_data)
route_colors_df

,노선,노선색
0,1호선,#003DA5
1,2호선,#83B600
2,3호선,#E51837
3,4호선,#009C3A
4,5호선,#FFD900
5,6호선,#8B50A4
6,7호선,#F05A28
7,8호선,#00A3E0
8,9호선,#AA9872
9,경의중앙선,#F5A200


In [38]:
seoul_metro_1to9_colors = ['darkblue', 'lightgreen', 'orange', 'lightblue', 'darkpurple',
                             'darkred', 'darkgreen', 'pink', 'lightgray']

## 기준 학원 함수 : folium_gasan_station(m)

In [39]:
def folium_gasan_station(m):
    folium.Marker([df_standard['위도'][1],df_standard['경도'][1]], 
                icon =folium.Icon(icon= 'star', prefix='fa',color=seoul_metro_1to9_colors [0])
                ).add_to(m)
    folium.Marker([df_standard['위도'][0],df_standard['경도'][0]], 
                icon =folium.Icon(icon= 'star', prefix='fa',color=seoul_metro_1to9_colors [6])
                ).add_to(m)


가산디지털단지와 데이터프레임 비교 정렬

In [40]:
df.sort_values(by='호선',inplace=True)
df_compare = df[df['역명']!='가산디지털단지']
df_compare.reset_index(drop=True, inplace=True)
df_compare



,역명,호선,위도,경도
0,서울역,1호선,37.556228,126.972135
1,종각,1호선,37.570161,126.982923
2,종로3가,1호선,37.570406,126.991847
3,종로5가,1호선,37.570926,127.001849
4,동대문,1호선,37.571687,127.010930
...,...,...,...,...
764,지평,중앙선,37.476393,127.629874
765,팔당,중앙선,37.547371,127.243939
766,진접,진접선,37.720500,127.203400
767,별내별가람,진접선,37.667780,127.115810


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 770 to 675
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      771 non-null    object 
 1   호선      771 non-null    object 
 2   위도      771 non-null    float64
 3   경도      771 non-null    float64
dtypes: float64(2), object(2)
memory usage: 30.1+ KB


In [42]:
df[df['호선']=='7호선(인천)']

,역명,호선,위도,경도
148,부평구청,7호선(인천),37.508480,126.720770
149,굴포천,7호선(인천),37.506810,126.732340
150,삼산체육관,7호선(인천),37.507240,126.741790
151,상동,7호선(인천),37.505814,126.753163
152,부천시청,7호선(인천),37.504440,126.763640
156,까치울,7호선(인천),37.506130,126.810930
155,부천종합운동장,7호선(인천),37.505020,126.796610
153,신중동,7호선(인천),37.502820,126.775660
154,춘의,7호선(인천),37.503650,126.788280


In [43]:
grouped = df.groupby('호선')

# 각 그룹에 대해 데이터프레임 생성
grouped_dfs = {}
for name, group in grouped:
    grouped_dfs[name] = group

line =[]
# 결과 확인
for name, df_group in grouped_dfs.items():
    line.append(name)  
line

['1호선',
 '2호선',
 '3호선',
 '4호선',
 '5호선',
 '6호선',
 '7호선',
 '7호선(인천)',
 '8호선',
 '9호선',
 '9호선(연장)',
 '경강선',
 '경부선',
 '경원선',
 '경의중앙선',
 '경인선',
 '경춘선',
 '공항철도1호선',
 '과천선',
 '김포골드라인',
 '분당선',
 '서해선',
 '수인선',
 '신림선',
 '신분당선',
 '신분당선(연장)',
 '신분당선(연장2)',
 '안산선',
 '에버라인선',
 '우이신설선',
 '의정부선',
 '인천1호선',
 '인천2호선',
 '일산선',
 '장항선',
 '중앙선',
 '진접선']

In [44]:
df['호선'] = df['호선'].str.split('(').str[0]
folium_seoul_metro_lines = sorted(df['호선'].unique())
folium_seoul_metro_lines

['1호선',
 '2호선',
 '3호선',
 '4호선',
 '5호선',
 '6호선',
 '7호선',
 '8호선',
 '9호선',
 '경강선',
 '경부선',
 '경원선',
 '경의중앙선',
 '경인선',
 '경춘선',
 '공항철도1호선',
 '과천선',
 '김포골드라인',
 '분당선',
 '서해선',
 '수인선',
 '신림선',
 '신분당선',
 '안산선',
 '에버라인선',
 '우이신설선',
 '의정부선',
 '인천1호선',
 '인천2호선',
 '일산선',
 '장항선',
 '중앙선',
 '진접선']

In [45]:
numlist=[]

for 호선 in folium_seoul_metro_lines:
    count = (df['호선'] == 호선).sum()
    print(f"{호선}은 {count}개")
    numlist.append(count)

1호선은 10개
2호선은 50개
3호선은 34개
4호선은 26개
5호선은 58개
6호선은 39개
7호선은 62개
8호선은 18개
9호선은 38개
경강선은 11개
경부선은 39개
경원선은 33개
경의중앙선은 31개
경인선은 20개
경춘선은 19개
공항철도1호선은 14개
과천선은 9개
김포골드라인은 10개
분당선은 35개
서해선은 15개
수인선은 18개
신림선은 11개
신분당선은 16개
안산선은 13개
에버라인선은 15개
우이신설선은 13개
의정부선은 15개
인천1호선은 30개
인천2호선은 27개
일산선은 11개
장항선은 7개
중앙선은 21개
진접선은 3개


In [46]:
#1호선 데이터 대부분이 누락되어 있음..?

#>> 서울교통공사에서 운영하는 호선만 데이터 있음
df[df['호선']=='1호선']

,역명,호선,위도,경도
770,서울역,1호선,37.556228,126.972135
768,종각,1호선,37.570161,126.982923
767,종로3가,1호선,37.570406,126.991847
766,종로5가,1호선,37.570926,127.001849
765,동대문,1호선,37.571687,127.010930
764,신설동,1호선,37.576048,127.024634
763,제기동,1호선,37.578103,127.034893
762,청량리,1호선,37.579956,127.044585
761,동묘앞,1호선,37.573197,127.016480
769,시청,1호선,37.565715,126.977088


In [47]:
# AWS INSTANCE에서 데이터 가져오기

yespa_db = mysql.connector.connect(
    host = 'database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'eda_team',
    database = 'eda_project',
    password = '***'
)

cur = yespa_db.cursor(buffered=True, dictionary=True)

cur.execute('SELECT * FROM seoul_metro_1line_coor')
result2 = cur.fetchall()

for data in result2:
    print(data)
    
yespa_db.close()

{'역명': '지행', '호선': '1호선', '위도': 37.892361, '경도': 127.055734}
{'역명': '도봉', '호선': '1호선', '위도': 37.679441, '경도': 127.045521}
{'역명': '창동', '호선': '1호선', '위도': 37.653181, '경도': 127.047663}
{'역명': '월계', '호선': '1호선', '위도': 37.633114, '경도': 127.058835}
{'역명': '외대앞', '호선': '1호선', '위도': 37.596157, '경도': 127.063572}
{'역명': '대방', '호선': '1호선', '위도': 37.513392, '경도': 126.926407}
{'역명': '신길', '호선': '1호선', '위도': 37.517018, '경도': 126.917743}
{'역명': '덕정', '호선': '1호선', '위도': 37.843216, '경도': 127.061511}
{'역명': '구일', '호선': '1호선', '위도': 37.496273, '경도': 126.869599}
{'역명': '온수', '호선': '1호선', '위도': 37.492267, '경도': 126.823399}
{'역명': '역곡', '호선': '1호선', '위도': 37.48514, '경도': 126.811622}
{'역명': '송내', '호선': '1호선', '위도': 37.487626, '경도': 126.753045}
{'역명': '부평', '호선': '1호선', '위도': 37.489457, '경도': 126.724584}
{'역명': '성균관대', '호선': '1호선', '위도': 37.300344, '경도': 126.970861}
{'역명': '소요산', '호선': '1호선', '위도': 37.5653, '경도': 127.0339}
{'역명': '동두천', '호선': '1호선', '위도': 37.927569, '경도': 127.054825}
{'역명': '보산', '호선': '1호선'

In [48]:
df_1line = pd.DataFrame(result2)
df_1line.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      99 non-null     object 
 1   호선      99 non-null     object 
 2   위도      99 non-null     float64
 3   경도      99 non-null     float64
dtypes: float64(2), object(2)
memory usage: 3.2+ KB


In [49]:
df.drop(df.loc[df['호선'] == '1호선'].index, axis=0, inplace=True)
df

,역명,호선,위도,경도
724,당산,2호선,37.534946,126.902767
725,영등포구청,2호선,37.525706,126.896610
726,문래,2호선,37.517933,126.894760
727,신도림,2호선,37.508961,126.891084
733,서울대입구,2호선,37.481247,126.952739
...,...,...,...,...
597,지평,중앙선,37.476393,127.629874
607,팔당,중앙선,37.547371,127.243939
676,진접,진접선,37.720500,127.203400
674,별내별가람,진접선,37.667780,127.115810


In [50]:
df_new = pd.concat([df,df_1line], ignore_index=True)
df_new

,역명,호선,위도,경도
0,당산,2호선,37.534946,126.902767
1,영등포구청,2호선,37.525706,126.896610
2,문래,2호선,37.517933,126.894760
3,신도림,2호선,37.508961,126.891084
4,서울대입구,2호선,37.481247,126.952739
...,...,...,...,...
855,동대문,1호선,37.565145,127.007890
856,신설동,1호선,37.576117,127.024710
857,제기동,1호선,37.578116,127.034902
858,청량리(서울시립대입구),1호선,37.580148,127.045063


In [51]:

df_compare = df_new[df_new['역명']!='가산디지털단지']
df_compare .sort_values(by='호선',inplace=True)
df_compare.reset_index(drop=True, inplace=True)
df_compare

/tmp/ipykernel_6145/2678176193.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compare .sort_values(by='호선',inplace=True)


,역명,호선,위도,경도
0,동묘앞,1호선,37.573265,127.016459
1,녹천,1호선,37.644599,127.051406
2,석계,1호선,37.615206,127.065594
3,신이문,1호선,37.602745,127.067556
4,회기,1호선,37.589802,127.057936
...,...,...,...,...
852,양정,중앙선,37.605330,127.193640
853,신원,중앙선,37.525545,127.372921
854,오남,진접선,37.705000,127.192810
855,진접,진접선,37.720500,127.203400


In [52]:
numlist=[]

for 호선 in folium_seoul_metro_lines:
    count = (df_new['호선'] == 호선).sum()
    print(count)
    numlist.append(count)

len(numlist)

99
50
34
26
58
39
62
18
38
11
39
33
31
20
19
14
9
10
35
15
18
11
16
13
15
13
15
30
27
11
7
21
3


33

In [53]:

import folium

m = folium.Map(location=[37.58, 127.0], zoom_start=10)

for i in range(0,sum(numlist[:1])):
    folium.Marker([df_compare['위도'][i],df_compare['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[0],
                                    icon='1',
                                    prefix='fa')
                  ).add_to(m)
for i in range(sum(numlist[:1]),sum(numlist[:2])):
    folium.Marker([df_compare['위도'][i],df_compare['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[1], 
                                    icon='2',
                                    prefix='fa')
                  ).add_to(m)   
for i in range(sum(numlist[:2]),sum(numlist[:3])):
    folium.Marker([df_compare['위도'][i],df_compare['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[2], 
                                    icon='3',
                                    prefix='fa')
                  ).add_to(m)
    
m

In [54]:
import folium

# Folium 맵 생성
m = folium.Map(location=[37.58, 127.0], zoom_start=10)

# 노선 수에 맞게 반복
for line_index in range(9):
    # 각 노선의 시작과 끝 인덱스 계산
    start_index = sum(numlist[:line_index])
    end_index = sum(numlist[:line_index + 1])
    
    # 해당 노선에 대한 마커 추가
    for i in range(start_index, end_index):
        folium.Marker(
            [df_compare['위도'][i], df_compare['경도'][i]], 
            icon=folium.Icon(
                color=seoul_metro_1to9_colors[line_index],
                icon=str(line_index + 1),
                prefix='fa'),
                popup =df_compare['역명'][i]
                
            
        ).add_to(m)
folium_gasan_station(m)
m


In [56]:
import folium

# Folium 맵 생성
m = folium.Map(location=[37.58, 127.0], zoom_start=10)

# 노선 수에 맞게 반복
for line_index in range(9):
    print(line_index)
    # 각 노선의 시작과 끝 인덱스 계산
    start_index = sum(numlist[:line_index])
    end_index = sum(numlist[:line_index + 1])
    print(f"Start index for line {line_index + 1}: {start_index}, End index: {end_index}")
    
    # 해당 노선에 대한 마커 추가
    for i in range(start_index, end_index):
        folium.Marker(
            [df_compare['위도'][i], df_compare['경도'][i]], 
            icon=folium.Icon(
                color=seoul_metro_1to9_colors[line_index],
                icon=str(line_index + 1),
                prefix='fa'
            )
        ).add_to(m)

0
Start index for line 1: 0, End index: 99
1
Start index for line 2: 99, End index: 149
2
Start index for line 3: 149, End index: 183
3
Start index for line 4: 183, End index: 209
4
Start index for line 5: 209, End index: 267
5
Start index for line 6: 267, End index: 306
6
Start index for line 7: 306, End index: 368
7
Start index for line 8: 368, End index: 386
8
Start index for line 9: 386, End index: 424


In [57]:
df_compare[df_compare['호선']=='9호선']

,역명,호선,위도,경도
384,등촌,9호선,37.550632,126.865689
385,올림픽공원,9호선,37.516269,127.130288
386,여의도,9호선,37.521760,126.924030
387,흑석,9호선,37.508770,126.963708
388,노량진,9호선,37.513534,126.941005
389,샛강,9호선,37.517274,126.928422
390,선정릉,9호선,37.510297,127.043999
391,둔촌오륜,9호선,37.519683,127.137989
392,한성백제,9호선,37.516404,127.116503
393,송파나루,9호선,37.510372,127.112216


In [58]:
dfex = df_new[df_new['역명'].isin(['삼산체육관','부평구청','굴포천'])]
dfex

,역명,호선,위도,경도
246,부평구청,7호선,37.507394,126.721599
247,굴포천,7호선,37.506997,126.731280
250,삼산체육관,7호선,37.506411,126.742153
260,부평구청,7호선,37.508480,126.720770
261,굴포천,7호선,37.506810,126.732340
262,삼산체육관,7호선,37.507240,126.741790
669,부평구청,인천1호선,37.508407,126.720555


In [59]:
df_refined=df_new.drop([260,261,262])
df_refined

,역명,호선,위도,경도
0,당산,2호선,37.534946,126.902767
1,영등포구청,2호선,37.525706,126.896610
2,문래,2호선,37.517933,126.894760
3,신도림,2호선,37.508961,126.891084
4,서울대입구,2호선,37.481247,126.952739
...,...,...,...,...
855,동대문,1호선,37.565145,127.007890
856,신설동,1호선,37.576117,127.024710
857,제기동,1호선,37.578116,127.034902
858,청량리(서울시립대입구),1호선,37.580148,127.045063


In [60]:
df_refined.sort_values(by='호선',inplace=True)
df_refined.reset_index(drop=True, inplace=True)
df_refined

,역명,호선,위도,경도
0,동묘앞,1호선,37.573265,127.016459
1,녹천,1호선,37.644599,127.051406
2,석계,1호선,37.615206,127.065594
3,신이문,1호선,37.602745,127.067556
4,회기,1호선,37.589802,127.057936
...,...,...,...,...
852,양정,중앙선,37.605330,127.193640
853,신원,중앙선,37.525545,127.372921
854,오남,진접선,37.705000,127.192810
855,진접,진접선,37.720500,127.203400


In [61]:
numlist_refined=[]

for 호선 in folium_seoul_metro_lines:
    count = (df_refined['호선'] == 호선).sum()
    print(count)
    numlist_refined.append(count)

len(numlist_refined)

99
50
34
26
58
39
59
18
38
11
39
33
31
20
19
14
9
10
35
15
18
11
16
13
15
13
15
30
27
11
7
21
3


33

In [62]:
import folium

# Folium 맵 생성
m = folium.Map(location=[37.58, 127.0], zoom_start=10)

# 노선 수에 맞게 반복
for line_index in range(9):
    # 각 노선의 시작과 끝 인덱스 계산
    start_index = sum(numlist_refined[:line_index])
    end_index = sum(numlist_refined[:line_index + 1])
    print(f"Start index for line {line_index + 1}: {start_index}, End index: {end_index}")
    
    # 해당 노선에 대한 마커 추가
    for i in range(start_index, end_index):
        folium.Marker(
            [df_refined['위도'][i],df_refined['경도'][i]], 
            icon=folium.Icon(
                color=seoul_metro_1to9_colors[line_index],
                icon=str(line_index + 1),
                prefix='fa'),
                popup =df_refined['역명'][i]
                
            
        ).add_to(m)
folium_gasan_station(m)
m

Start index for line 1: 0, End index: 99
Start index for line 2: 99, End index: 149
Start index for line 3: 149, End index: 183
Start index for line 4: 183, End index: 209
Start index for line 5: 209, End index: 267
Start index for line 6: 267, End index: 306
Start index for line 7: 306, End index: 365
Start index for line 8: 365, End index: 383
Start index for line 9: 383, End index: 421


In [63]:
df_refined[df_refined['호선']=='4호선']

,역명,호선,위도,경도
183,노원,4호선,37.656270,127.063276
184,상계,4호선,37.660878,127.073572
185,명동,4호선,37.560989,126.986325
186,회현,4호선,37.558514,126.978246
187,서울역,4호선,37.552810,126.972556
188,삼각지,4호선,37.534075,126.972600
189,신용산,4호선,37.529170,126.967894
190,사당,4호선,37.476955,126.981651
191,동작,4호선,37.502852,126.980347
192,총신대입구,4호선,37.486263,126.981989
